In [ ]:
source("../shared.R")

In [ ]:
dat = read.csv(file="pctgrowth.csv", header=TRUE)

In [ ]:
summary(dat)

In [ ]:
table(dat$grp)

In [ ]:
boxplot( y ~ grp, data=dat)

In [ ]:
str_mod = "model {
    for (i in 1:length(y)) {
        y[i] ~ dnorm(theta[grp[i]], prec)
    }
    for (j in 1:max(grp)) {
        theta[j] ~ dnorm(mu, tau_prec)
    }
    mu ~ dnorm(0, 1e-6)
    tau_prec ~ dgamma(1./2, 3./2)
    prec ~ dgamma(2./2, 2./2)

    sig = sqrt(1/prec)
    tau = sqrt(1/tau_prec)
}"

data_jags = as.list(dat)
mod = jags_model_compile(str_mod, data_jags)
params = c("theta", "mu", "sig", "tau")
mod_sim = jags_model_sample(mod, var_names_ = params, n_iter_ = 1e4)
mod_diag = jags_model_diag(mod_sim$samples)

In [ ]:
plot(mod_sim$samples)

In [ ]:
post_means = colMeans(mod_sim$samples_cmb)

In [ ]:
mean(dat$y)

In [ ]:
var(dat$y)

In [ ]:
summary(mod_sim$samples_cmb)

In [ ]:
post_means

In [ ]:
mean(mod_sim$samples_cmb[,'sig'] / mod_sim$samples_cmb[,'tau'] > 1)

## Anova analysis

In [ ]:
theta_means = post_means[4:length(post_means)]

In [ ]:
means_anova = tapply(dat$y, INDEX=dat$grp, FUN=mean)
plot(means_anova)
points(post_means[4:8], col="red")